"This notebook explores how an AI system can decide whether to respond, ask for clarification, defer, or stay silent."

## Decision Vocabulary
This system does not generate answers by default.
It first decides whether the AI should speak at all, based on usefulness, confidence, and context.

RESPOND

The AI is confident
The response adds value
Speaking now is appropriate
Example intuition:
The AI has enough information and the user clearly wants an answer.

ASK_CLARIFY
The intent is unclear or
Key information is missing
Speaking now may mislead
Example intuition:
Answering without clarification could be wrong or premature.

DEFER
The question is valid
But requires human judgment, authority, or accountability
Example intuition:
The AI should not take responsibility for this decision.

SILENT
The AI has low confidence or
The response would be noise or
Interrupting the user is not worth it
Example intuition:
Saying nothing is better than saying something low-quality.

In [1]:
from enum import Enum

class Decision(Enum):
    RESPOND = "respond"
    ASK_CLARIFY = "ask_clarify"
    DEFER = "defer"
    SILENT = "silent"


## Decision Signals (v1)
1️⃣ confidence_score

Range: 0.0 – 1.0

What it means:

How confident the system is that it can give a correct and grounded response.

2️⃣ clarity_score

Range: 0.0 – 1.0

What it means:

How clear the user’s intent is.

3️⃣ attention_cost

Range: 0.0 – 1.0

What it means:

How disruptive or unnecessary it would be to speak right now.

In [2]:
def decide_action(confidence_score, clarity_score, attention_cost):
    """
    Decide whether the AI should respond, ask for clarification,
    defer to a human, or stay silent.
    """

    # Low clarity → ask clarifying question
    if clarity_score < 0.4:
        return Decision.ASK_CLARIFY

    # Very low confidence → silence is better than noise
    if confidence_score < 0.3:
        return Decision.SILENT

    # Medium confidence but high responsibility → defer
    if 0.3 <= confidence_score < 0.6:
        return Decision.DEFER

    # High confidence but high attention cost → stay silent
    if confidence_score >= 0.6 and attention_cost > 0.7:
        return Decision.SILENT

    # Otherwise, respond
    return Decision.RESPOND


In [3]:
test_cases = [
    ("Clear & confident", 0.9, 0.9, 0.2),
    ("Unclear intent", 0.8, 0.2, 0.1),
    ("Low confidence", 0.2, 0.8, 0.1),
    ("Medium confidence", 0.5, 0.8, 0.2),
    ("High confidence but noisy", 0.9, 0.9, 0.9),
]

for name, conf, clar, cost in test_cases:
    decision = decide_action(conf, clar, cost)
    print(f"{name}: {decision.value}")


Clear & confident: respond
Unclear intent: ask_clarify
Low confidence: silent
Medium confidence: defer
High confidence but noisy: silent


## Signal Estimation (v1 – Heuristic)
In v1, signals are estimated using simple heuristics. Later versions may use LLMs, retrieval, or historical data.

In [4]:
def estimate_clarity(user_query):
    """
    Estimate how clear the user's intent is.
    Very naive heuristic for v1.
    """
    if len(user_query.strip()) < 10:
        return 0.2
    if "?" not in user_query:
        return 0.5
    return 0.8


In [5]:
def estimate_confidence(user_query):
    """
    Placeholder confidence estimation.
    In later versions, this will be based on
    retrieval similarity or model uncertainty.
    """
    keywords = ["how", "what", "explain"]
    if any(k in user_query.lower() for k in keywords):
        return 0.7
    return 0.4


In [6]:
def estimate_attention_cost(user_query, context=None):
    """
    Estimate how disruptive or unnecessary a response would be.
    """
    if context and context.get("user_busy"):
        return 0.8
    if "urgent" in user_query.lower():
        return 0.2
    return 0.5


In [7]:
def should_ai_speak(user_query, context=None):
    clarity = estimate_clarity(user_query)
    confidence = estimate_confidence(user_query)
    attention_cost = estimate_attention_cost(user_query, context)

    decision = decide_action(confidence, clarity, attention_cost)

    return {
        "query": user_query,
        "signals": {
            "clarity": clarity,
            "confidence": confidence,
            "attention_cost": attention_cost
        },
        "decision": decision.value
    }


In [8]:
examples = [
    "Explain Kubernetes",
    "Help",
    "Should we deploy now?",
    "URGENT: system down",
    ""
]

for q in examples:
    print(should_ai_speak(q, context={"user_busy": False}))


{'query': 'Explain Kubernetes', 'signals': {'clarity': 0.5, 'confidence': 0.7, 'attention_cost': 0.5}, 'decision': 'respond'}
{'query': 'Help', 'signals': {'clarity': 0.2, 'confidence': 0.4, 'attention_cost': 0.5}, 'decision': 'ask_clarify'}
{'query': 'Should we deploy now?', 'signals': {'clarity': 0.8, 'confidence': 0.4, 'attention_cost': 0.5}, 'decision': 'defer'}
{'query': 'URGENT: system down', 'signals': {'clarity': 0.5, 'confidence': 0.4, 'attention_cost': 0.2}, 'decision': 'defer'}
{'query': '', 'signals': {'clarity': 0.2, 'confidence': 0.4, 'attention_cost': 0.5}, 'decision': 'ask_clarify'}


In [9]:
knowledge_base = [
    "Kubernetes is a container orchestration system used to deploy, scale, and manage containerized applications.",
    "Production deployments should be handled carefully and often require human approval.",
    "System outages are critical incidents and should be handled by on-call engineers."
]


In [10]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

kb_embeddings = model.encode(knowledge_base)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

def estimate_confidence(user_query):
    query_embedding = model.encode([user_query])
    similarities = cosine_similarity(query_embedding, kb_embeddings)
    max_similarity = similarities.max()
    return float(max_similarity)


In [12]:
for q in examples:
    print(should_ai_speak(q, context={"user_busy": False}))


{'query': 'Explain Kubernetes', 'signals': {'clarity': 0.5, 'confidence': 0.789440393447876, 'attention_cost': 0.5}, 'decision': 'respond'}
{'query': 'Help', 'signals': {'clarity': 0.2, 'confidence': 0.1361997276544571, 'attention_cost': 0.5}, 'decision': 'ask_clarify'}
{'query': 'Should we deploy now?', 'signals': {'clarity': 0.8, 'confidence': 0.5208808183670044, 'attention_cost': 0.5}, 'decision': 'defer'}
{'query': 'URGENT: system down', 'signals': {'clarity': 0.5, 'confidence': 0.4805813431739807, 'attention_cost': 0.2}, 'decision': 'defer'}
{'query': '', 'signals': {'clarity': 0.2, 'confidence': 0.1084282249212265, 'attention_cost': 0.5}, 'decision': 'ask_clarify'}


In [13]:
def estimate_confidence(user_query):
    query_embedding = model.encode([user_query])
    similarities = cosine_similarity(query_embedding, kb_embeddings)[0]

    max_similarity = similarities.max()

    return {
        "confidence": float(max_similarity),
        "similarities": similarities
    }


In [14]:
def estimate_confidence(user_query):
    query_embedding = model.encode([user_query])
    similarities = cosine_similarity(query_embedding, kb_embeddings)[0]

    max_similarity = similarities.max()

    return {
        "confidence": float(max_similarity),
        "similarities": similarities
    }


In [15]:
def should_ai_speak(user_query, context=None):
    clarity = estimate_clarity(user_query)

    conf_data = estimate_confidence(user_query)
    confidence = conf_data["confidence"]

    attention_cost = estimate_attention_cost(user_query, context)

    decision = decide_action(confidence, clarity, attention_cost)

    return {
        "query": user_query,
        "signals": {
            "clarity": clarity,
            "confidence": confidence,
            "attention_cost": attention_cost
        },
        "decision": decision.value
    }


In [16]:
tests = [
    "Explain Kubernetes networking",
    "Explain it",
    "What should we do?",
    "asdfghjkl"
]

for t in tests:
    print(should_ai_speak(t, context={"user_busy": False}))


{'query': 'Explain Kubernetes networking', 'signals': {'clarity': 0.5, 'confidence': 0.7277780175209045, 'attention_cost': 0.5}, 'decision': 'respond'}
{'query': 'Explain it', 'signals': {'clarity': 0.5, 'confidence': 0.025051362812519073, 'attention_cost': 0.5}, 'decision': 'silent'}
{'query': 'What should we do?', 'signals': {'clarity': 0.8, 'confidence': 0.22431394457817078, 'attention_cost': 0.5}, 'decision': 'silent'}
{'query': 'asdfghjkl', 'signals': {'clarity': 0.2, 'confidence': 0.07128375768661499, 'attention_cost': 0.5}, 'decision': 'ask_clarify'}


In [17]:
def act_on_decision(result):
    decision = result["decision"]

    if decision == "respond":
        return "AI should generate a response."

    if decision == "ask_clarify":
        return "AI should ask a clarifying question."

    if decision == "defer":
        return "AI should escalate this to a human."

    if decision == "silent":
        return "AI should not respond."


In [18]:
for t in tests:
    result = should_ai_speak(t, context={"user_busy": False})
    action = act_on_decision(result)
    print(t, "→", result["decision"], "→", action)


Explain Kubernetes networking → respond → AI should generate a response.
Explain it → silent → AI should not respond.
What should we do? → silent → AI should not respond.
asdfghjkl → ask_clarify → AI should ask a clarifying question.


In [19]:
from transformers import pipeline

llm = pipeline(
    "text-generation",
    model="gpt2",
    max_new_tokens=50
)

def generate_response(query):
    output = llm(query)[0]["generated_text"]
    return output


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [20]:
def act_on_decision(result):
    decision = result["decision"]
    query = result["query"]

    if decision == "respond":
        return generate_response(query)

    if decision == "ask_clarify":
        return "Can you clarify what you mean?"

    if decision == "defer":
        return "This requires human judgment. Escalating."

    if decision == "silent":
        return None


In [21]:
def decide_action(confidence, clarity, attention_cost):

    if clarity < 0.4:
        return "ask_clarify", "User intent is unclear"

    if confidence < 0.3:
        return "silent", "No reliable knowledge available"

    if 0.3 <= confidence < 0.6:
        return "defer", "Confidence is not high enough for AI to decide"

    if confidence >= 0.6 and attention_cost > 0.7:
        return "silent", "High interruption cost"

    return "respond", "High confidence and clear intent"


In [22]:
def decide_action(confidence, clarity, attention_cost):

    if clarity < 0.4:
        return "ask_clarify", "User intent is unclear"

    if confidence < 0.3:
        return "silent", "No reliable knowledge available"

    if 0.3 <= confidence < 0.6:
        return "defer", "Confidence is not high enough for AI to decide"

    if confidence >= 0.6 and attention_cost > 0.7:
        return "silent", "High interruption cost"

    return "respond", "High confidence and clear intent"


In [24]:
def should_ai_speak(user_query, context=None):
    clarity = estimate_clarity(user_query)

    conf_data = estimate_confidence(user_query)
    confidence = conf_data["confidence"]

    attention_cost = estimate_attention_cost(user_query, context)

    decision, reason = decide_action(confidence, clarity, attention_cost)

    return {
        "query": user_query,
        "signals": {
            "clarity": clarity,
            "confidence": confidence,
            "attention_cost": attention_cost
        },
        "decision": decision,
        "reason": reason
    }


In [25]:
def act_on_decision(result):
    decision = result["decision"]
    reason = result["reason"]
    query = result["query"]

    if decision == "respond":
        return generate_response(query)

    if decision == "ask_clarify":
        return f"Clarification needed: {reason}"

    if decision == "defer":
        return f"Escalated: {reason}"

    if decision == "silent":
        return None


In [26]:
for t in tests:
    result = should_ai_speak(t, context={"user_busy": False})
    print(result)


{'query': 'Explain Kubernetes networking', 'signals': {'clarity': 0.5, 'confidence': 0.7277780175209045, 'attention_cost': 0.5}, 'decision': 'respond', 'reason': 'High confidence and clear intent'}
{'query': 'Explain it', 'signals': {'clarity': 0.5, 'confidence': 0.025051362812519073, 'attention_cost': 0.5}, 'decision': 'silent', 'reason': 'No reliable knowledge available'}
{'query': 'What should we do?', 'signals': {'clarity': 0.8, 'confidence': 0.22431394457817078, 'attention_cost': 0.5}, 'decision': 'silent', 'reason': 'No reliable knowledge available'}
{'query': 'asdfghjkl', 'signals': {'clarity': 0.2, 'confidence': 0.07128375768661499, 'attention_cost': 0.5}, 'decision': 'ask_clarify', 'reason': 'User intent is unclear'}
